# HW2 - Pandas and ICD-codes

### Get the data

For this assignment, we'll need to get some data! We will be using the Diabetes Dataset that is located here:

https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008#

Afterwards, unzip the folder and place the contents in a folder called `/data/` at this directory.

Make sure to add a file called `.gitignore` at the root of your directory and add the line 

```
    data/
```
to it so that it ignore any files that you place in the Data folder.

In [51]:
import pandas as pd
import numpy as np
import string
import re

In [52]:
diabetes_df = pd.read_csv('./data/diabetic_data.csv')

![](data/datadictionary.png)

## Data Source [5 pts]

When we begin working with data, it is important to try and understand the data that we've been given. Often the context can tell us a lot of information about the data. In fact, often times understanding what is *not* in the data is just as critical as understanding the data itself. From the above link, the information about how the data was collected can be found in this [paper](https://www.hindawi.com/journals/bmri/2014/781670/)


List the 5 inclusion criteria to be in the dataset: 

1. It is an inpatient encounter (a hospital admission).
2. It is a “diabetic” encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.
3. The length of stay was at least 1 day and at most 14 days.
4. Laboratory tests were performed during the encounter.
5. Medications were administered during the encounter.


## Explore the Data [55 pts]
Using the data, answer the following questions:

#### 1. How many rows does the `diabetes_df` have? How many columns? [2.5 pts]

In [11]:
#rows = len(diabetes_df['patient_nbr'])
#rows

In [12]:
#columns = len(diabetes_df.isnull().sum())
#columns

In [53]:
rows = len(diabetes_df)
columns = len (list(diabetes_df))
rows, columns

(101766, 50)

#### 2. How many unique encounters are there? How many unique patients? [2.5 pts]

In [54]:
uni_encounters = diabetes_df ['encounter_id'].unique()
len(uni_encounters)

101766

In [55]:
uni_patients = diabetes_df ['patient_nbr'].unique()
len(uni_patients)

71518

#### 3. What is the most amount of encounters that a single patient has in the dataset? [5 pts]

In [56]:
from collections import Counter
c = Counter (diabetes_df['patient_nbr'])
c.most_common(1)

[(88785891, 40)]


#### 4. Show the proportion of non-missing values are in the dataset for each column. [10 pts]
> Make sure you check to see if there are missing values that aren't coded as missing, but should be

In [57]:
diabetes_df = diabetes_df.replace('?', np.NaN)
diabetes_df = diabetes_df.replace('NA', np.NaN)
missing_sum = diabetes_df.isnull().sum()
non_missisng_proportion = ( rows - missing_sum ) / rows
non_missisng_proportion

# no_info
# invalid
# unknown
# ?
# python treats NA automatically as NaN

encounter_id                1.000000
patient_nbr                 1.000000
race                        0.977664
gender                      1.000000
age                         1.000000
weight                      0.031415
admission_type_id           1.000000
discharge_disposition_id    1.000000
admission_source_id         1.000000
time_in_hospital            1.000000
payer_code                  0.604426
medical_specialty           0.509178
num_lab_procedures          1.000000
num_procedures              1.000000
num_medications             1.000000
number_outpatient           1.000000
number_emergency            1.000000
number_inpatient            1.000000
diag_1                      0.999794
diag_2                      0.996482
diag_3                      0.986017
number_diagnoses            1.000000
max_glu_serum               1.000000
A1Cresult                   1.000000
metformin                   1.000000
repaglinide                 1.000000
nateglinide                 1.000000
c

#### 5. For all numeric columns, show summary statistics (mean, median, max, min, etc) [2.5 pts]

In [58]:
numeric_columns = diabetes_df.select_dtypes(include=['int64'])
numeric_columns.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


#### 6. For all columns with a `dtype` of object, show the count of all of the values in that column [5 pts]
> You may want to reference the `pd.DataFrame.select_dtypes()` function 

In [59]:
# object_columns = diabetes_df.select_dtypes(include=['object'])

[diabetes_df[c].value_counts() 
 for c in list(diabetes_df.select_dtypes(include=['object']).columns)]

[Caucasian          76099
 AfricanAmerican    19210
 Hispanic            2037
 Other               1506
 Asian                641
 Name: race, dtype: int64, Female             54708
 Male               47055
 Unknown/Invalid        3
 Name: gender, dtype: int64, [70-80)     26068
 [60-70)     22483
 [50-60)     17256
 [80-90)     17197
 [40-50)      9685
 [30-40)      3775
 [90-100)     2793
 [20-30)      1657
 [10-20)       691
 [0-10)        161
 Name: age, dtype: int64, [75-100)     1336
 [50-75)       897
 [100-125)     625
 [125-150)     145
 [25-50)        97
 [0-25)         48
 [150-175)      35
 [175-200)      11
 >200            3
 Name: weight, dtype: int64, MC    32439
 HM     6274
 SP     5007
 BC     4655
 MD     3532
 CP     2533
 UN     2448
 CM     1937
 OG     1033
 PO      592
 DM      549
 CH      146
 WC      135
 OT       95
 MP       79
 SI       55
 FR        1
 Name: payer_code, dtype: int64, InternalMedicine                     14635
 Emergency/Trauma          

#### 7. What is the average number of labs administered by age category [2.5 pts]

In [60]:
diabetes_df.groupby('age').mean()[['num_lab_procedures']]

,num_lab_procedures
age,
[0-10),41.012422
[10-20),43.096961
[20-30),43.066385
[30-40),43.033642
[40-50),42.785958
[50-60),42.611961
[60-70),42.600632
[70-80),43.157396
[80-90),44.085015


#### 8. Does the number of diagnoses equal the number of non-NA entries in the diag_* columns? [2.5 pts]

In [61]:
diabetes_df['number_diagnoses'].sum()

755369

In [62]:
s = diabetes_df['diag_1'].isnull().sum() + diabetes_df['diag_2'].isnull().sum() + diabetes_df['diag_3'].isnull().sum()
rows*3-s


303496

They are not equal.

#### 9. Create a new column that has the value of 1 if the medical specialty in that row contains the word Surgery and 0 otherwise. [10 pts]

In [73]:
diabetes_df = pd.read_csv('./data/diabetic_data.csv')

In [74]:
diabetes_df['new_column'] = 0

for word in ['Surgery']:
    diabetes_df['new_column'][diabetes_df['medical_specialty'].str.contains(word)] = 1
                              
#diabetes_df['new_column']



/Users/megancui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [75]:
diabetes_df[['medical_specialty','new_column']]

,medical_specialty,new_column
0,Pediatrics-Endocrinology,0
1,?,0
2,?,0
3,?,0
4,?,0
5,?,0
6,?,0
7,?,0
8,?,0
9,InternalMedicine,0


#### 10. How many encounters where the patient was between the ages of 0 and 20 took place with Pediatric providers? [5 pts]

In [76]:
#diabetes_df[diabetes_df['medical_specialty'].str.contains('Pediatric') & (diabetes_df['age'] == "[0-10)")]

df = diabetes_df[diabetes_df['medical_specialty'].str.contains('Pediatric') & (diabetes_df['age'].isin(['[0-10)', '[10-20)']))]
df['encounter_id'].count()

429

#### 11. Join in the admission source, admission_type_id, and discharge_disposition_id to the diabetes dataframe. Use the default inner join method [5 pts]

In [682]:
# will NOT need to complete 

#### 12. find the average A1C (if available) for patients that were readmitted vs. not readmitted patients [2.5 pts]

In [683]:
# change this to: 
# find the counts of each of the available A1Cresult categories, 
# broken down by whether or not the patient was readmitted 
# (regardless of whether it was less than or greater than 30 days)

In [77]:
df1 = diabetes_df[(diabetes_df['readmitted'].isin(['<30', '>30']))]
df1['A1Cresult'].value_counts()

None    39426
>8       3712
Norm     2081
>7       1683
Name: A1Cresult, dtype: int64

In [78]:
df2 = diabetes_df[(diabetes_df['readmitted'] == "NO")]
df2['A1Cresult'].value_counts()

None    45322
>8       4504
Norm     2909
>7       2129
Name: A1Cresult, dtype: int64


## Single-level CCS categories [40 pts]

The columns `diag_1`, `diag_2`,  and `diag_3` contain ICD-9-CM codes for the encounters that took place in this dataset. However, if we count up the number of unique values between the 3 columns, we can see that the data is very sparse. 

As we discussed in class, the single-level CCS categories can be used instead to group similar ICD codes together. Download the latest version of the ICD-9-CM single-level CCS here: [https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip](https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip)

Unzip this and put the contents in the `data` folder along with the Diabetes dataset and read the file called `$dxref 2015.csv` into a variable. **NOTE** You must skip the first row of this `csv` file when reading it in because there is a note there that is unrelated to the contents of the data. Look up how to do this using the `pd.read_csv` function

In [79]:
df = pd.read_csv('./data/$dxref 2015.csv',skiprows = 1)
df

,'ICD-9-CM CODE','CCS CATEGORY','CCS CATEGORY DESCRIPTION','ICD-9-CM CODE DESCRIPTION','OPTIONAL CCS CATEGORY','OPTIONAL CCS CATEGORY DESCRIPTION'
0,' ','0 ','No DX',INVALID CODES IN USER DATA,' ',' '
1,'01000','1 ','Tuberculosis',PRIM TB COMPLEX-UNSPEC,' ',' '
2,'01001','1 ','Tuberculosis',PRIM TB COMPLEX-NO EXAM,' ',' '
3,'01002','1 ','Tuberculosis',PRIM TB COMPLEX-EXM UNKN,' ',' '
4,'01003','1 ','Tuberculosis',PRIM TB COMPLEX-MICRO DX,' ',' '
5,'01004','1 ','Tuberculosis',PRIM TB COMPLEX-CULT DX,' ',' '
6,'01005','1 ','Tuberculosis',PRIM TB COMPLEX-HISTO DX,' ',' '
7,'01006','1 ','Tuberculosis',PRIM TB COMPLEX-OTH TEST,' ',' '
8,'01010','1 ','Tuberculosis',PRIM TB PLEURISY-UNSPEC,' ',' '
9,'01011','1 ','Tuberculosis',PRIM TB PLEURISY-NO EXAM,' ',' '


Examine the contents of the data. Please bear in mind that Jupyter notebooks do not render whitespace or quotes very well sometimes, so watch out for that. Make sure you examine column names with `.columns` instead of just calling `.head()` and visually inspecting, for example.

In [687]:
df.columns

Index([''ICD-9-CM CODE'', ''CCS CATEGORY'', ''CCS CATEGORY DESCRIPTION'',
       ''ICD-9-CM CODE DESCRIPTION'', ''OPTIONAL CCS CATEGORY'',
       ''OPTIONAL CCS CATEGORY DESCRIPTION''],
      dtype='object')

If you notice, the ICD codes are not very well-formatted in either the Diabetes dataset (for example, the code `8` should really be `008.0`) or the Single-level CCS crosswalk (`' and whitespace characters`). 

This is quite typical of healthcare data, unfortunately. Many of the publically available files are not suited for reading into modern programming languages. Often, they are limited to SAS format datasets, which is a proprietary software suite for working with statistical packages that is ubiquitous in health care.

In order to use the CCS groupings, we'll have to clean both the groupings *and* the diabetes data as well. Here is the general procedure that we will take (Although this is an imperfect mapping as well).

**Remove all quotes and extra whitespace from the codes and the column names in the CCS crosswalk [10 pts]**

In [80]:
df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.replace("'", "")

df1 = df.applymap(lambda x: x.replace("'", ""))
new_df = df1.applymap(lambda x: x.replace(' ', ''))
new_df

,ICD-9-CMCODE,CCSCATEGORY,CCSCATEGORYDESCRIPTION,ICD-9-CMCODEDESCRIPTION,OPTIONALCCSCATEGORY,OPTIONALCCSCATEGORYDESCRIPTION
0,,0,NoDX,INVALIDCODESINUSERDATA,,
1,01000,1,Tuberculosis,PRIMTBCOMPLEX-UNSPEC,,
2,01001,1,Tuberculosis,PRIMTBCOMPLEX-NOEXAM,,
3,01002,1,Tuberculosis,PRIMTBCOMPLEX-EXMUNKN,,
4,01003,1,Tuberculosis,PRIMTBCOMPLEX-MICRODX,,
5,01004,1,Tuberculosis,PRIMTBCOMPLEX-CULTDX,,
6,01005,1,Tuberculosis,PRIMTBCOMPLEX-HISTODX,,
7,01006,1,Tuberculosis,PRIMTBCOMPLEX-OTHTEST,,
8,01010,1,Tuberculosis,PRIMTBPLEURISY-UNSPEC,,
9,01011,1,Tuberculosis,PRIMTBPLEURISY-NOEXAM,,


Next, we're going to write a function that cleans up the ICD codes found in the Diabetes dataset. If you'll notice, there are no decimal points in the single-level CCS crosswalk. Therefore, we must make sure that our data matches that as well. Implement the function below, and we will use it in an `.apply()` call to modify the `diag_` columns.

#### Implement this function [15 pts]

In [81]:
diabetes_df = pd.read_csv('./data/diabetic_data.csv')

In [82]:
def clean_diabetes_code(icd_code):
    """
    Formats codes found in the Diabetes dataset to be like those found in the CCS crosswalk
    
    
    If an icd_code has a decimal, remove the decimal
    
    if it has less than 3 digits, prepend '0's, until it is 3 digits.
    
    If it has 3 digits (before or after the above step), append a '0'
    
    Examples:
        250.13 -> 25013
        32 -> 0320
        315 -> 3150
        
    Args: 
        icd_code:
    
    Returns:
        formatted_code string: A formatted ICD-Code string according to the above criteria:
        
        
    """
    code_copy = str(icd_code)  # Convert to string due to weird .apply behavior in Series
    
    code_copy = code_copy.replace('?','nan')
    code_copy = code_copy.replace(".", "")
    
    if code_copy == 'nan':
        return np.nan
    
    ### Your Code here: -- modify code_copy until it returns the reformatted code. 
    ### Store this in formatted_code and return it
    
    else:
        if len(code_copy) <= 3:
            formatted_code = code_copy.zfill(3)
            formatted_code = formatted_code.ljust(4, '0')
            return formatted_code
        
        if len(code_copy) >3:
            formatted_code = code_copy
            return formatted_code
        
   
    
    

Now, replace all three `diag_` columns by calling .apply with this function. For example:

`diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)`

In [83]:

diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)
diabetes_df['diag_2'] = diabetes_df['diag_2'].apply(clean_diabetes_code)
diabetes_df['diag_3'] = diabetes_df['diag_3'].apply(clean_diabetes_code)

# filter_col = [col 
#               for col in diabetes_df 
#               if col.startswith('diag_')]

# diabetes_df[filter_col]

diag_df = diabetes_df[['diag_1','diag_2','diag_3']]
diag_df

,diag_1,diag_2,diag_3
0,25083,NaN,NaN
1,2760,25001,2550
2,6480,2500,V270
3,0080,25043,4030
4,1970,1570,2500
5,4140,4110,2500
6,4140,4110,V450
7,4280,4920,2500
8,3980,4270,0380
9,4340,1980,4860


In [84]:
diag_df_1 = diag_df['diag_1']
diag_df_1 = pd.DataFrame(diag_df_1,columns=['diag_1'])

diag_df_2 = diag_df['diag_2']
diag_df_2 = pd.DataFrame(diag_df_2,columns=['diag_2'])

diag_df_3 = diag_df['diag_3']
diag_df_3 = pd.DataFrame(diag_df_3,columns=['diag_3'])

#### Join in the single-level CCS crosswalk and answer the following question: [15 pts]

List the top 10 condition categories (`CCS CATEGORY DESCRIPTION`) when you add up all instances over all 3 columns

In [712]:
# top_code = [diabetes_df[col].value_counts() 
#             for col in diabetes_df 
#             if col.startswith('diag_')]
# top_code


In [711]:
# df3 = df2[(df2['ICD-9-CMCODE'] == "4280")]

# df3['CCSCATEGORYDESCRIPTION'].value_counts()

In [85]:
df_3 = pd.merge(diag_df_3,
                new_df[['ICD-9-CMCODE','CCSCATEGORY','CCSCATEGORYDESCRIPTION']],
                left_on = 'diag_3',
                right_on = 'ICD-9-CMCODE',
                how='left')

In [86]:
df_2 = pd.merge(diag_df_2,
                new_df[['ICD-9-CMCODE','CCSCATEGORY','CCSCATEGORYDESCRIPTION']],
                left_on = 'diag_2',
                right_on = 'ICD-9-CMCODE',
                how='left')

In [87]:
df_1 = pd.merge(diag_df_1,
                new_df[['ICD-9-CMCODE','CCSCATEGORY','CCSCATEGORYDESCRIPTION']],
                left_on = 'diag_1',
                right_on = 'ICD-9-CMCODE',
                how='left')

In [88]:
ccd = df_1['CCSCATEGORYDESCRIPTION'].value_counts()+df_2['CCSCATEGORYDESCRIPTION'].value_counts()+df_3['CCSCATEGORYDESCRIPTION'].value_counts()

In [89]:
top_ccd = ccd.sort_values(ascending = False)
top_ccd.head(10)

Htncomplicn    18653.0
chf;nonhp      18101.0
Coronathero    17602.0
Fluid/elcdx    13816.0
Dysrhythmia    12762.0
DiabMelw/cm    10266.0
UTI             7039.0
Anemia          5068.0
COPD            4867.0
Coma/brndmg     4844.0
Name: CCSCATEGORYDESCRIPTION, dtype: float64